<a href="https://colab.research.google.com/github/nephelim74/PhoneStoreETL/blob/main/DZ2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score
import numpy as np


In [2]:
# Загрузка датасета
df = pd.read_csv('fifa_s2.csv')
# Просмотр первых строк датафрейма
df.head()


,ID,Name,Age,Nationality,Overall,Potential,Club,Value,Wage,Preferred Foot,International Reputation,Skill Moves,Position,Joined,Contract Valid Until,Height,Weight,Release Clause
0,1179,G. Buffon,40,Italy,88,88,Paris Saint-Germain,4000.0,77,Right,4.0,1.0,GK,2018,01/01/2019,6.333333,203.0,7400.0
1,5479,Casillas,37,Spain,82,82,FC Porto,1500.0,10,Left,4.0,1.0,NaN,2015,01/01/2019,6.083333,185.0,3000.0
2,8205,Braga,35,Portugal,70,70,CD Aves,750.0,5,Right,1.0,3.0,LCM,2017,01/01/2019,5.916667,165.0,1600.0
3,9014,A. Robben,34,Netherlands,84,84,FC Bayern München,15500.0,110,Left,4.0,4.0,RM,2009,01/01/2019,5.916667,176.0,25600.0
4,19521,J. Ibehre,35,England,61,61,Cambridge United,120.0,2,Right,1.0,2.0,ST,2017,01/01/2019,6.333333,203.0,210.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2399 entries, 0 to 2398
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID                        2399 non-null   int64  
 1   Name                      2399 non-null   object 
 2   Age                       2399 non-null   int64  
 3   Nationality               2392 non-null   object 
 4   Overall                   2399 non-null   int64  
 5   Potential                 2399 non-null   int64  
 6   Club                      2158 non-null   object 
 7   Value                     2142 non-null   float64
 8   Wage                      2399 non-null   int64  
 9   Preferred Foot            2399 non-null   object 
 10  International Reputation  2389 non-null   float64
 11  Skill Moves               2393 non-null   float64
 12  Position                  2378 non-null   object 
 13  Joined                    2399 non-null   int64  
 14  Contract

In [4]:

# Проверка на наличие пропущенных значений
print(df.isnull().sum())



ID                            0
Name                          0
Age                           0
Nationality                   7
Overall                       0
Potential                     0
Club                        241
Value                       257
Wage                          0
Preferred Foot                0
International Reputation     10
Skill Moves                   6
Position                     21
Joined                        0
Contract Valid Until        273
Height                        0
Weight                        0
Release Clause                3
dtype: int64


In [5]:
# Удаление ненужных колонок (если такие есть)
# Например, если колонка 'Unnamed: 0' существует, удалим её
df = df.drop(columns=['Unnamed: 0'], errors='ignore')

In [6]:
# Заполнение пропусков в числовых колонках медианными значениями
numeric_columns = ['Value', 'Wage', 'International Reputation', 'Skill Moves', 'Release Clause']
df[numeric_columns] = df[numeric_columns].fillna(df[numeric_columns].median())

In [7]:
# Проверка на наличие пропущенных значений
print(df.isnull().sum())

ID                            0
Name                          0
Age                           0
Nationality                   7
Overall                       0
Potential                     0
Club                        241
Value                         0
Wage                          0
Preferred Foot                0
International Reputation      0
Skill Moves                   0
Position                     21
Joined                        0
Contract Valid Until        273
Height                        0
Weight                        0
Release Clause                0
dtype: int64


In [8]:
# Заполнение пропусков в категориальных колонках модой
categorical_columns = ['Nationality', 'Position', 'Preferred Foot', 'Contract Valid Until']
df[categorical_columns] = df[categorical_columns].fillna(df[categorical_columns].mode().iloc[0])


In [9]:
# Проверка на наличие пропущенных значений
print(df.isnull().sum())

ID                            0
Name                          0
Age                           0
Nationality                   0
Overall                       0
Potential                     0
Club                        241
Value                         0
Wage                          0
Preferred Foot                0
International Reputation      0
Skill Moves                   0
Position                      0
Joined                        0
Contract Valid Until          0
Height                        0
Weight                        0
Release Clause                0
dtype: int64


Доля пропусков в колонке Club составляет около 10%, что попадает в категорию умеренных пропусков. Это означает, что удаление строк с пропущенными значениями может быть не самым оптимальным решением, так как мы потеряем около 10% данных.
Попробуем использовать модель машинного обучения для предсказания пропущенных значений для данного поля

In [10]:
# Разделение данных на обучающую и тестовую выборки
# Обучающая выборка: строки с известными значениями 'Club'
train_data = df[df['Club'].notna()]
# Тестовая выборка: строки с пропущенными значениями 'Club'
test_data = df[df['Club'].isna()]

In [11]:
# Выбор признаков для модели
features = ['Age', 'Nationality', 'Overall', 'Potential', 'Value', 'Wage', 'Position', 'International Reputation', 'Skill Moves']
X_train = train_data[features]
y_train = train_data['Club']
X_test = test_data[features]

In [12]:
# Проверка наличия столбцов в X_train
print("Столбцы в X_train:", X_train.columns)

Столбцы в X_train: Index(['Age', 'Nationality', 'Overall', 'Potential', 'Value', 'Wage',
       'Position', 'International Reputation', 'Skill Moves'],
      dtype='object')


In [13]:
# Если столбец 'Preferred Foot' отсутствует, удаляем его из списка категориальных признаков
if 'Preferred Foot' not in X_train.columns:
    categorical_features = ['Nationality', 'Position']  # Убираем 'Preferred Foot'
else:
    categorical_features = ['Nationality', 'Position', 'Preferred Foot']

In [14]:
# Создаем ColumnTransformer для кодирования категориальных признаков
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough'  # Оставляем числовые признаки без изменений
)

In [15]:
# Применяем преобразование к обучающей и тестовой выборкам
X_train_encoded = preprocessor.fit_transform(X_train)
X_test_encoded = preprocessor.transform(X_test)


In [16]:
# Разделение обучающей выборки на тренировочную и валидационную части
X_train_split, X_val, y_train_split, y_val = train_test_split(X_train_encoded, y_train, test_size=0.2, random_state=42)

In [17]:
# Обучение модели (Random Forest)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_split, y_train_split)

RandomForestClassifier(random_state=42)

In [18]:
# Предсказание на валидационной выборке
y_pred = model.predict(X_val)

In [19]:
# Оценка точности модели
accuracy = accuracy_score(y_val, y_pred)
print(f"\nТочность модели на валидационной выборке: {accuracy:.2f}")


Точность модели на валидационной выборке: 0.23


In [22]:
# После неудачных попыток улучшить модель:
if accuracy < 0.5:
    print("Точность модели недостаточна. Заполняем пропуски модой.")
    most_frequent_club = df["Club"].mode()[0]
    df["Club"] = df["Club"].fillna(most_frequent_club)

Точность модели недостаточна. Заполняем пропуски модой.


In [24]:
# Проверка на наличие пропущенных значений после обработки
print("\nПропущенные значения после обработки:")
print(df.isnull().sum())


Пропущенные значения после обработки:
ID                          0
Name                        0
Age                         0
Nationality                 0
Overall                     0
Potential                   0
Club                        0
Value                       0
Wage                        0
Preferred Foot              0
International Reputation    0
Skill Moves                 0
Position                    0
Joined                      0
Contract Valid Until        0
Height                      0
Weight                      0
Release Clause              0
dtype: int64


In [25]:
# Приведение всех строковых данных к нижнему регистру
df = df.apply(lambda x: x.str.lower() if x.dtype == "object" else x)

In [28]:
# Удаление полных дубликатов
df = df.drop_duplicates()

In [30]:
# Проверка на наличие дубликатов после обработки
print("\nКоличество дубликатов после обработки:", df.duplicated().sum())


Количество дубликатов после обработки: 0


In [31]:
# Функция для разбиения возраста по группам
def age_group(age):
    if age < 20:
        return 'до 20'
    elif 20 <= age < 30:
        return 'от 20 до 30'
    elif 30 <= age < 36:
        return 'от 30 до 36'
    else:
        return 'старше 36'

In [33]:
# Добавление новой колонки с группами возраста
df['age_group'] = df['Age'].apply(age_group)

In [34]:
# Подсчет количества футболистов в каждой категории
age_group_counts = df['age_group'].value_counts()

In [35]:
# Вывод результата
print("\nКоличество футболистов в каждой возрастной группе:")
print(age_group_counts)


Количество футболистов в каждой возрастной группе:
age_group
от 20 до 30    1646
от 30 до 36     458
до 20           270
старше 36        23
Name: count, dtype: int64
